In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()

In [2]:
edges_12 = edges.filter(lambda e: e[0] == 12).cache()

In [3]:
def separate_num(x):
    return x[0]

d = 1
res_str = ''
e_12 = edges.filter(lambda e: e[1] == 12).cache()
is_34 = e_12.filter(lambda e: e[0] == 34).cache().count()
if is_34 > 0:
    res_str = res_str + '12,34' 
    print res_str;
else:
    e_12_first = e_12.map(separate_num)
    e_12_c = e_12_first.collect()
    prev_in_loop = e_12.persist()
    temp_in_loop = edges.filter(lambda e: e[1] in e_12_c).persist()
    join = edges.filter(lambda e: e[1] in e_12_c).map(lambda e: (e[1], e[0])).join(prev_in_loop) \
        .map(lambda (x,y) : (y[0], (x, y[1]))).persist()

    while (True):
        e_12_first = temp_in_loop.map(separate_num).persist()
        e_12_c = e_12_first.collect()
        temp_in_loop = edges.filter(lambda e: e[1] in e_12_c).persist()
        join = temp_in_loop.map(lambda e: (e[1], e[0])).join(join) \
        .map(lambda (x,y) : (y[0], (x, y[1]))).persist()
        is_34 = temp_in_loop.filter(lambda e: e[0] == 34).cache().count()
        if is_34 > 0:
            result = join.filter(lambda e: e[0] == 34).take(1)
            break;
        d = d+1
        prev_in_loop = temp_in_loop.persist()

    res_str = res_str + str(result[0][0])
    result = result[0][1]
    while True:
        try:
            Ptarget = int(result[1])
            res_str = res_str + ',' + str(result[0]) + ',' + str(result[1])
            break;
        except:
            res_str = res_str + ',' + str(result[0])
            result = result[1]

In [4]:
vertex = res_str.split(",")

i = len(vertex)
j = 2
final_str = '12'
while (j < i + 1):
    final_str = final_str + ',' +  vertex[i-j]
    j = j + 1

print final_str

12,422,53,52,107,20,23,274,34
